<a href="https://colab.research.google.com/github/MaryamNourii/Intent-detection-and-Slot-filling/blob/main/BERT_intent_Slot_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers

In [ ]:
!pip install nlpaug

In [ ]:
!unzip -qx dataset.zip -d dataset

In [ ]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel,TFBertForSequenceClassification

import tensorflow as tf
from keras import layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from transformers import BertModel, BertTokenizer, BertConfig

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow_addons.optimizers import AdamW
from sklearn.metrics import classification_report

In [ ]:
# MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
# MODEL_NAME_OR_PATH = "bert-base-multilingual-uncased"
MODEL_NAME_OR_PATH =  "HooshvareLab/bert-fa-zwnj-base"

In [ ]:
import pandas as pd
import nlpaug.augmenter.word as naw

data = pd.read_csv("/content/dataset/train.csv")

def augment_data(sentence, slots, intent_label, aug):
    augmented_sentence = aug.augment(sentence)
    return augmented_sentence, slots, intent_label

aug = naw.ContextualWordEmbsAug(model_path=MODEL_NAME_OR_PATH, action="insert")

aug_data = []
for i in range(len(data)):
    sentence = data.loc[i, 'sentence']
    slots = data.loc[i, 'slots']
    intent_label = data.loc[i, 'intent_label']
    augmented_sentence, slots, intent_label = augment_data(sentence, slots, intent_label, aug)
    aug_data.append([''.join(augmented_sentence), slots, intent_label])

aug_data = pd.DataFrame(aug_data, columns=['sentence', 'slots', 'intent_label'])

augmented_data = pd.concat([data, aug_data], ignore_index=True)
augmented_data=augmented_data.drop('index',axis=1)
augmented_data = augmented_data.sample(frac=1).reset_index(drop=True)
augmented_data.to_csv('augmented_intent_detection_data.csv', index=False)

,sentence,slots,intent_label
0,تاریخ کارت اولیه به عنوان کارت دو میخوام را از...,B-Method I-Method I-Method O O O B-Bankname O ...,transaction_card
1,او میخوام ، 50 تن هزار دلار تومن به علی باقری ...,O B-Amount I-Amount I-Amount O B-Username I-Us...,transaction_card
2,میخواهم پولی را را از حساب ريیس ملت پهلوی با ش...,O O O O O B-Bankname O O B-Accountnumber O O B...,transaction_card
3,همه میخوام هم به همین حساب علی منصور احمدی از ...,O O O B-Username I-Username O O B-Bankname B-M...,transaction_card
4,شما میخوام به حساب میرزا محمد میرزا توکلی پول ...,O O O B-Username I-Username O B-Method I-Metho...,transaction_card
...,...,...,...
108,امام علی بن اکبریم,B-Username I-Username,introduce
109,ترجمه محمد علی باقری ، هستم,B-Username I-Username O,introduce
110,« اسم خود من مولانا سجاد فضلی هست,O O B-Username I-Username O,introduce
111,احمد حسین میری زاده هستم,B-Username I-Username O,introduce
